In [1]:
import os
import math
import pandas as pd
from tqdm import tqdm
from kaggle_datasets import KaggleDatasets
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.backend as K

2022-06-21 03:57:16.281584: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-06-21 03:57:16.281739: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.0.0.2:8470


2022-06-21 03:57:21.219286: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-21 03:57:21.222205: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-06-21 03:57:21.222235: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-21 03:57:21.222262: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (0b1b96f83c6b): /proc/driver/nvidia/version does not exist
2022-06-21 03:57:21.224921: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

REPLICAS:  8


In [4]:
BATCH_SIZE=32 * strategy.num_replicas_in_sync
IMAGE_SIZE=[512,512]

In [5]:
def int_feature(int_list):
    if isinstance(int_list,type(tf.cast(1,dtype=tf.int32))):
        int_list=int_list.numpy()
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[int_list]))

def bytes_feature(value):
    if isinstance(value,type(tf.cast(1,dtype=tf.float32))):
        value=value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [6]:
def Searlize_to_string(image_feature,label_feature):
    feature={
        "image":image_feature,
        "label":label_feature
    }
    example=tf.train.Example(features=tf.train.Features(feature=feature))
    return example.SerializeToString()

In [7]:
#!python EDA.py --data_path train.csv --images_base_path train_images
#!python create_fold.py --data_path "processed_data/cleaned_data.csv"

In [8]:
df=pd.read_csv("../input/shop-tpu-folds/fold_data.csv")

In [9]:
df.head()

,posting_id,image,image_phash,title,label_group,image_path,gfold
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,666,train_images\0000a68812bc7e98c42888dfb1c07da0.jpg,0
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",7572,train_images\00039780dfc94d01db8676fe789ecd05.jpg,2
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,6172,train_images\000a190fdd715a2a36faed16e2c65df7.jpg,0
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,10509,train_images\00117e4fc239b1b641ff08340b429633.jpg,1
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,9425,train_images\00136d1cf4edede0203f32f05f660588.jpg,3


In [10]:
train_steps={}
valid_steps={}
for fold in range(5):
    valid_data=df.loc[df['gfold']==fold].reset_index(drop=True)
    valid_steps[fold]=valid_data.shape[0]
    train_steps[fold]=df.shape[0]-valid_data.shape[0]

In [11]:
train_steps,valid_steps

({0: 27400, 1: 27400, 2: 27400, 3: 27400, 4: 27400},
 {0: 6850, 1: 6850, 2: 6850, 3: 6850, 4: 6850})

In [12]:
#import os
#os.mkdir("tfrecords")

In [13]:
#for fold in tqdm(range(5)):
#    fold_data=df.loc[df['gfold']==fold].reset_index(drop=True)
#    with tf.io.TFRecordWriter(f"tfrecords/fold_{fold}_512.tfrec") as writer:
#        for i in fold_data.index:
#            img=tf.keras.preprocessing.image.load_img(fold_data.loc[i,'image_path'],target_size=IMAGE_SIZE)
#            img=tf.keras.preprocessing.image.img_to_array(img)
#            img = cv2.imencode('.jpg', img, (cv2.IMWRITE_JPEG_QUALITY, 94))[1].tostring()
#            img_bytes_feature=bytes_feature(img)
#            label_feature=int_feature(int(fold_data.loc[i,'label_group']))
#            example_string=Searlize_to_string(img_bytes_feature,label_feature)
#            writer.write(example_string)

In [14]:
GCS_PATH = KaggleDatasets().get_gcs_path('shop-512-size-tfrecords')

# DATA AUG

https://www.kaggle.com/code/cdeotte/rotation-augmentation-gpu-tpu-0-96/notebook

In [15]:
def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.
    
    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )
        
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape( tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3] )    
    
    # ZOOM MATRIX
    zoom_matrix = tf.reshape( tf.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3] )
    
    # SHIFT MATRIX
    shift_matrix = tf.reshape( tf.concat([one,zero,height_shift, zero,one,width_shift, zero,zero,one],axis=0),[3,3] )
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), K.dot(zoom_matrix, shift_matrix))

In [16]:
def SHIFT_ROTATE_ZOOM(image,label):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = IMAGE_SIZE[0]
    XDIM = DIM%2 #fix for size 331
    
    rot = 15. * tf.random.normal([1],dtype='float32')
    shr = 5. * tf.random.normal([1],dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    w_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    h_shift = 16. * tf.random.normal([1],dtype='float32') 
    w_shift = 16. * tf.random.normal([1],dtype='float32') 
  
    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image,tf.transpose(idx3))
        
    return tf.reshape(d,[DIM,DIM,3]),label

In [17]:
def Desearlize(example):
    ex={
        "image":tf.io.FixedLenFeature([],dtype=tf.string),
        "label":tf.io.FixedLenFeature([],dtype=tf.int64)
    }
    features=tf.io.parse_single_example(example,ex)
    image,label=features['image'],features['label']
    label=tf.cast(label,dtype=tf.int32)
    image=tf.image.decode_jpeg(image,channels=3)
    image=tf.image.resize(image,IMAGE_SIZE)
    image=tf.cast(image,dtype=tf.float32)/255.0
    ex={"image":image,"label":label}
    return ex

In [18]:
def APPLY_AUG(example):
    image,label=example['image'],example['label']
    #image,label=SHIFT_ROTATE_ZOOM(image,label)
    image=tf.image.random_flip_left_right(image,)
    image=tf.image.random_brightness(image,0.2)
    image=tf.image.random_contrast(image,0.02,1)
    return image,label

In [19]:
def Train_input(image,label):
    return (image,label),label
def Valid_input(example):
    image,label=example['image'],example['label']
    return (image,label),label

In [20]:
def Valid_data_pipe(files):
    AUTO=tf.data.experimental.AUTOTUNE
    dataset = tf.data.TFRecordDataset(files, num_parallel_reads = AUTO)
    dataset=dataset.map(Desearlize,num_parallel_calls=AUTO)
    dataset=dataset.map(Valid_input,num_parallel_calls=AUTO)
    dataset=dataset.batch(BATCH_SIZE,drop_remainder=True)
    dataset=dataset.prefetch(AUTO)
    return dataset

def Train_data_pipe(files):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    AUTO=tf.data.experimental.AUTOTUNE
    dataset = tf.data.TFRecordDataset(files, num_parallel_reads = AUTO )
    dataset=dataset.with_options(ignore_order)
    dataset=dataset.map(Desearlize,num_parallel_calls=AUTO)
    dataset=dataset.map(APPLY_AUG,num_parallel_calls=AUTO)
    dataset=dataset.map(Train_input,num_parallel_calls=AUTO)
    dataset = dataset.repeat()
    dataset=dataset.shuffle(1024)
    dataset=dataset.batch(BATCH_SIZE)
    dataset=dataset.prefetch(AUTO)
    return dataset

In [21]:
files=tf.io.gfile.glob(f"{GCS_PATH}/tfrecords/fold_*.tfrec")

2022-06-21 03:58:35.310697: I tensorflow/core/platform/cloud/google_auth_provider.cc:180] Attempting an empty bearer token since no token was retrieved from files, and GCE metadata check was skipped.


In [22]:
from tensorflow.keras.applications import EfficientNetB5

In [23]:
class ARCFACE_LAYER(tf.keras.layers.Layer):
    def __init__(self,m=0.5,s=30,n_classes=11014):
        super(ARCFACE_LAYER,self).__init__()
        self.m=m
        self.s=s
        self.sin_m=tf.sin(m)
        self.cos_m=tf.cos(m)
        self.n_classes=n_classes
        self.threshold = tf.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def build(self,input_shape):
        prev_layer_units=input_shape[0][1]
        self.w=self.add_weight(shape=(prev_layer_units,self.n_classes),trainable=True,
                              initializer='glorot_uniform')

    def get_config(self):
        config=super().get_config()
        config.update({"m":0.5,
                       "s":30,
                       "n_classes":11014})
        return config


    def call(self,inputs):
        prev_layer,y=inputs
        y=tf.cast(y,dtype=tf.int32)
        y_hot=tf.one_hot(y,self.n_classes)
        y_hot=tf.cast(y_hot,dtype=tf.float32)
        w_norm=tf.linalg.l2_normalize(self.w,axis=0)
        x_norm=tf.linalg.l2_normalize(prev_layer,axis=1)
        cos_theta=tf.linalg.matmul(x_norm,w_norm)
        sin_theta=tf.sqrt(1-tf.pow(cos_theta,tf.cast(2,dtype=tf.float32)))
        cos_theta_m=(cos_theta*self.cos_m)-(sin_theta*self.sin_m)
        cos_theta_m=tf.where(cos_theta>self.threshold,cos_theta_m,cos_theta-self.mm)
        final=self.s*((y_hot*cos_theta_m)+((1-y_hot)*cos_theta))
        return final

In [24]:
LR_START = 0.000005
LR_MAX   = 0.001#0.00005 * strategy.num_replicas_in_sync
LR_MIN   = 0.000005
LR_RAMPUP_EPOCHS = 5
LR_EXP_DECAY = .8

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS) + LR_MIN
    return lr

In [25]:
for fold in range(5):
    train_records=files[:fold]+files[fold+1:]
    test_records=files[fold]
    NUM_TRAIN_STEPS=train_steps[fold]//BATCH_SIZE
    NUM_VALID_STEPS=valid_steps[fold]//BATCH_SIZE
    tf.keras.backend.clear_session()
    with strategy.scope():
        pre_trained=EfficientNetB5(include_top=False,weights="imagenet",input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
        ins=tf.keras.layers.Input(())
        x=pre_trained.layers[-1].output
        x=tf.keras.layers.GlobalMaxPooling2D()(x)
        #x=tf.keras.layers.Dense(720)(x)
        arc_layer=ARCFACE_LAYER()
        x=arc_layer([x,ins])
        outs=tf.keras.layers.Softmax()(x)
        model=tf.keras.models.Model(inputs=(pre_trained.input,ins),outputs=outs)
        print(f"training for fold {fold}")
        #print(model.summary())
        model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    lr_callback=tf.keras.callbacks.LearningRateScheduler(lambda epoch: lrfn(epoch),verbose=1)
    early=tf.keras.callbacks.EarlyStopping(monitor="val_loss",mode="min",verbose=1,patience=10)
    saver=tf.keras.callbacks.ModelCheckpoint(filepath="effb5"+f"{fold}.h5",
                                     monitor="val_loss",mode="min",save_best_only=True,
                                     save_weights_only=True,verbose=1)
    train_dataloader=Train_data_pipe(train_records)
    valid_dataloader=Valid_data_pipe(test_records)
    model.fit(train_dataloader,
              validation_data=valid_dataloader,epochs=40,
              callbacks=[early,saver,lr_callback],steps_per_epoch=NUM_TRAIN_STEPS,
             validation_steps=NUM_VALID_STEPS)
    print(f"model training for {fold} is done")
    del model
    import gc
    gc.collect()

115269632/115263384 [==============================] - 1s 0us/step
training for fold 0
Epoch 1/40

Epoch 00001: LearningRateScheduler reducing learning rate to 5e-06.
107/107 [==============================] - 277s 1s/step - loss: 23.8726 - sparse_categorical_accuracy: 0.0000e+00 - val_loss: 23.8225 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00001: val_loss improved from inf to 23.82248, saving model to effb50.h5
Epoch 2/40

Epoch 00002: LearningRateScheduler reducing learning rate to 0.00020400000000000003.
107/107 [==============================] - 131s 1s/step - loss: 23.7551 - sparse_categorical_accuracy: 0.0000e+00 - val_loss: 23.4719 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00002: val_loss improved from 23.82248 to 23.47189, saving model to effb50.h5
Epoch 3/40

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00040300000000000004.
107/107 [==============================] - 132s 1s/step - loss: 23.0785 - sparse_categorical_accuracy: 0.0000e+00 

In [ ]:
#Experiments 5 fold error
with dense
1) B0->1e-5 to 4e-4 -> 12.648266
   B0->1e-6 to 4e-5 -> "very high error"
   B0 ->1e-6 to 4e-4 -> 12.823738
without dense
1) B0->1e-5 to 4e-4 -> 13.708846
##################################################
(only 1 fold)
no dense
1) B0->1e-5 to 4e-4->13.8519
2) B0-> 1e-5 to 1e-3->11.1301
   B0-> 1e-6 to 1e-3->11.08619
With dense
1) B0->1e-5 to 4e-4-> 12.53761
2) B0-> 1e-5 to 1e-3->10.94927
   B0-> 1e-6 to 1e-3->10.72708(30 epochs)
###############################################
B3(dense)
1) B3 -> 1e-6 to 1e-3 -> 10.26380(50 epochs)
2) B3 -> 5e-6 to 1e-3 -> 10.11745(44 epochs)
B3(no dense)
5e-6 to 1e-3->10.06651(40 epochs)
1e-6 to 1e-3 -> 10.17261
1e-5 to 4e-4 -> 11.89102
#may be during training I observed that in b3 case dense layer is causing overfiitting at first.
##########################################
B5(no dense)
5e-6 to 1e-3 -> 9.59691
B5(no dense removed shift scale rotate)
5 fold error
5e-6 to 1e-3 ->9.31131

In [ ]:
""